In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.metrics import silhouette_score
import umap
from wordcloud import WordCloud
from gensim.models.coherencemodel import CoherenceModel


In [11]:
news_data = pd.read_csv('guardian_articles.csv')
news_data.head(5)


,article_id,sectionName,webTitle,webUrl,bodyContent,webPublicationDate,id
0,us-news/2016/jan/31/iowa-caucus-underdog-candi...,US news,Iowa underdogs put on brave faces despite all ...,https://www.theguardian.com/us-news/2016/jan/3...,As polling day looms and the cameras turn only...,2016-01-31T23:53:37Z,1
1,us-news/2016/jan/31/iowa-caucus-worlds-most-pa...,US news,Iowa caucus: hologram eagle and Jesus star on ...,https://www.theguardian.com/us-news/2016/jan/3...,"In Des Moines on Sunday, the Guardian was give...",2016-01-31T23:46:28Z,2
2,world/2016/jan/31/tanzania-britsh-helicopter-p...,World news,British pilot in Tanzania 'manoeuvred ​to save...,https://www.theguardian.com/world/2016/jan/31/...,A British pilot who was shot dead by an elepha...,2016-01-31T23:43:48Z,3
3,football/2016/jan/31/late-winner-gets-usa-off-...,Football,USA 3-2 Iceland | International friendly match...,https://www.theguardian.com/football/2016/jan/...,USA took a step toward shaking off the ghosts ...,2016-01-31T23:30:49Z,4
4,football/2016/jan/31/blackburn-paul-lambert-ox...,Football,Reinvigorated Paul Lambert reflects after impr...,https://www.theguardian.com/football/2016/jan/...,"The clean-shaven, spectacle free and suspiciou...",2016-01-31T22:30:10Z,5


In [12]:
news_data.describe()

,id
count,149839.00000
mean,74920.00000
std,43254.93783
min,1.00000
25%,37460.50000
50%,74920.00000
75%,112379.50000
max,149839.00000


In [13]:
news_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149839 entries, 0 to 149838
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   article_id          149839 non-null  object
 1   sectionName         149839 non-null  object
 2   webTitle            149839 non-null  object
 3   webUrl              149839 non-null  object
 4   bodyContent         148731 non-null  object
 5   webPublicationDate  149839 non-null  object
 6   id                  149839 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 8.0+ MB


In [14]:
news_data.drop(columns=["article_id","webUrl","webPublicationDate","id"],inplace=True)

In [15]:
stemmer = PorterStemmer()
stop_words = stopwords.words('english')

In [51]:
def remove_single_letters(sentence):
    words = sentence.split()
    filtered_words = [word for word in words if len(word) > 1 or not word.isalpha()]
    return ' '.join(filtered_words)

def preprocess(text):
    tokens = word_tokenize(remove_single_letters(text.lower()))
    filtered_tokens = [stemmer.stem(token) for token in tokens if (token not in stop_words and token.isalpha())]
    return filtered_tokens

num_sections = news_data['sectionName'].nunique()
sampled_index = np.random.choice(news_data.index, size=20000, replace=False)

sampled_news_data = news_data.iloc[sampled_index]


sampled_news_data=sampled_news_data[sampled_news_data['bodyContent'].apply(lambda x: not isinstance(x, float))]
sampled_news_data['tokens'] = sampled_news_data['bodyContent'].apply(preprocess)

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

class AutoencoderModule(nn.Module):
    def __init__(self, input_dim, latent_dim, activation):
        super(AutoencoderModule, self).__init__()
        if activation == 'relu':
            self.activation = nn.ReLU()
        # Extend for other activations if needed
        
        self.encoder = nn.Linear(input_dim, latent_dim)
        self.decoder = nn.Linear(latent_dim, input_dim)
        
    def forward(self, x):
        x = self.activation(self.encoder(x))
        x = self.activation(self.decoder(x))
        return x

class Autoencoder:
    def __init__(self, latent_dim=32, activation='relu', epochs=200, batch_size=128):
        self.latent_dim = latent_dim
        self.activation = activation
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = None
        self.loss_fn = nn.MSELoss()
        self.optimizer = None
        self.his = []

    def fit(self, X):
        if self.model is None:
            self.model = AutoencoderModule(X.shape[1], self.latent_dim, self.activation)
            self.optimizer = optim.Adam(self.model.parameters())
        
        dataset = torch.utils.data.TensorDataset(torch.tensor(X, dtype=torch.float32))
        loader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        
        X_train, X_test = train_test_split(X, test_size=0.2)
        
        train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32))
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        
        test_dataset = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.float32))
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)
        
        for epoch in range(self.epochs):
            for batch in train_loader:
                inputs = batch[0]
                outputs = self.model(inputs)
                loss = self.loss_fn(outputs, inputs)
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
            # Validation loss
            with torch.no_grad():
                val_loss = sum(self.loss_fn(self.model(inputs), inputs) for inputs, in test_loader)
                self.his.append(val_loss / len(test_loader))


In [95]:
import gensim
from gensim import corpora
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import numpy as np

class TopicEmbedder:
    def __init__(self, num_topics=8, gamma=90, latent_dim=32, n_clusters=8):
        self.num_topics = num_topics
        self.gamma = gamma
        self.dictionary = None
        self.lda_model = None
        self.bert_model = SentenceTransformer('distilbert-base-nli-mean-tokens')
        self.autoencoder = Autoencoder(latent_dim=latent_dim)
        self.kmeans = KMeans(n_clusters=n_clusters)
        self.latent_rep = {}
        self.corpus = None
        self.cluster_number = n_clusters
        
    def lda_embed(self, token_lists):
        if not self.dictionary:
            self.dictionary = corpora.Dictionary(token_lists)
        self.corpus = [self.dictionary.doc2bow(tokens) for tokens in token_lists]
        
        if not self.lda_model:
            self.lda_model = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.num_topics, id2word=self.dictionary,passes=20)

        def get_vec_lda(model, corpus, k):
                """
                Get the LDA vector representation (probabilistic topic assignments for all documents)
                :return: vec_lda with dimension: (n_doc * n_topic)
                """
                n_doc = len(corpus)
                vec_lda = np.zeros((n_doc, k))
                for i in range(n_doc):
                    # get the distribution for the i-th document in corpus
                    for topic, prob in model.get_document_topics(corpus[i]):
                        vec_lda[i, topic] = prob

                return vec_lda

        vec = get_vec_lda(self.lda_model, self.corpus, self.cluster_number)
        print('Getting vector representations for LDA. Done!')
        return vec
        
    def bert_embed(self, sentences):
        return self.bert_model.encode(sentences)
    
    def transform(self, sentences, token_lists, fit=True):
        lda_embeddings = self.lda_embed(token_lists)
        bert_embeddings = self.bert_embed(sentences)
        combined_embeddings = combined_embeddings = np.c_[np.array(lda_embeddings) * self.gamma, np.array(bert_embeddings)]
        
        # Fit the autoencoder
        if fit:
            self.autoencoder.fit(combined_embeddings)

        # Extract latent representation using the encoder part of the autoencoder
        with torch.no_grad():
            latent_representation = self.autoencoder.model.encoder(torch.tensor(combined_embeddings, dtype=torch.float32))
        
        return latent_representation.numpy()
    def fit(self, sentences, token_lists):
        self.latent_rep["train"] = self.transform(sentences, token_lists)
        self.kmeans.fit(self.latent_rep["train"])
        return self.kmeans.labels_

    def predict(self, sentences, token_lists):
        self.latent_rep["predict"] = self.transform(sentences, token_lists)
        cluster_labels = self.kmeans.predict(self.latent_rep["predict"])
        return cluster_labels


embedder = TopicEmbedder()
labels = embedder.fit(sampled_news_data["bodyContent"].tolist(), sampled_news_data["tokens"].tolist())


Getting vector representations for LDA. Done!


/Users/hosein/anaconda3/envs/tp/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [80]:

def get_topic_words(token_lists, labels):
    k = len(np.unique(labels))
    topics = ['' for _ in range(k)]
    for i, c in enumerate(token_lists):
        topics[labels[i]] += (' ' + ' '.join(c))
    word_counts = list(map(lambda x: Counter(x.split()).items(), topics))
    # get sorted word counts
    word_counts = list(map(lambda x: sorted(x, key=lambda x: x[1], reverse=True), word_counts))
    # get topics
    topics = list(map(lambda x: list(map(lambda x: x[0], x[:10])), word_counts))

    return topics

def get_coherence(model, token_lists, measure='c_v'):
    topics = get_topic_words(token_lists, model.kmeans.labels_)
    cm = CoherenceModel(topics=topics, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary,
                            coherence=measure)
    return cm.get_coherence()

def get_silhouette(model):
    lbs = model.kmeans.labels_
    vec = model.latent_rep['train']
    return silhouette_score(vec, lbs)

def plot_proj(embedding, lbs):
    n = len(embedding)
    counter = Counter(lbs)
    for i in range(len(np.unique(lbs))):
        plt.plot(embedding[:, 0][lbs == i], embedding[:, 1][lbs == i], '.', alpha=0.5,
                 label='cluster {}: {:.2f}%'.format(i, counter[i] / n * 100))
    plt.legend(loc = 'best')
    plt.grid(color ='grey', linestyle='-',linewidth = 0.25)


def visualize(embeddings, lbs):
    reducer = umap.UMAP()
    print('Calculating UMAP projection ...')
    vec_umap = reducer.fit_transform(embeddings)
    print('Calculating UMAP projection. Done!')
    plot_proj(vec_umap,lbs)

def get_wordcloud(embeddings, lbs, token_lists, topic):
    lbs = lbs
    tokens = ' '.join([' '.join(_) for _ in np.array(token_lists)[lbs == topic]])

    wordcloud = WordCloud(width=800, height=560,
                          background_color='white', collocations=False,
                          min_font_size=10).generate(tokens)

    plt.figure(figsize=(8, 5.6), facecolor=None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)

In [96]:
print(get_topic_words(sampled_news_data["tokens"].tolist(),labels))
print(get_coherence(embedder,sampled_news_data["tokens"].tolist()))
print(get_silhouette(embedder))

[['said', 'year', 'compani', 'would', 'uk', 'busi', 'govern', 'also', 'new', 'bank'], ['one', 'like', 'year', 'say', 'time', 'peopl', 'work', 'new', 'make', 'first'], ['game', 'play', 'one', 'player', 'team', 'said', 'time', 'year', 'first', 'leagu'], ['said', 'would', 'trump', 'parti', 'govern', 'peopl', 'vote', 'one', 'labour', 'elect'], ['said', 'one', 'us', 'peopl', 'year', 'use', 'also', 'countri', 'would', 'say'], ['said', 'peopl', 'govern', 'would', 'year', 'new', 'health', 'also', 'say', 'one'], ['peopl', 'said', 'say', 'one', 'year', 'like', 'time', 'work', 'would', 'get'], ['said', 'govern', 'polic', 'would', 'peopl', 'year', 'report', 'also', 'one', 'offic']]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | fals

In [97]:
sampled_news_data["labels"] = labels

In [98]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

sampled_news_data.head(50)


sectionName                                                                                             webTitle  \
109187             Opinion                     Labour's target should be the Tory party, not Johnson's credibility | Owen Jones   
121948         Environment                                                      Idaho bill seeks to kill more than 1,000 wolves   
88292               Cities                                  'An indictment of South Africa': whites-only town Orania is booming   
133203  Television & radio                                       Benedict Cumberbatch to play poisoned Soviet spy in HBO series   
142611          World news             ‘Pick the shelves clean’: food shortage rap helps cut through gloom of Shanghai lockdown   
66369                Music                                      Maren Morris review – acoustic warmth from country chart-topper   
12997                Books                             Real Swallows and Amazons children were refugees fleeing Syria | Letters   
868                  Music                          Bloc Party: Hymns review – modernised sound marred by ham-fisted sensuality   
1713              Football                              Sunderland hoping to hijack Aston Villa’s loan move for Mathieu Debuchy   
119802               Books                                              Poem of the Week: Hymn to Aphrodite by Frederick Seidel   
28621              UK news                                           London attack fuels calls for tighter Westminster security   
130302      Australia news                         Morrison government is paying PR firm $2.9m to promote Covid vaccine rollout   
141246      Australia news               ‘Actively develop friendship’: China’s new ambassador to Australia strikes softer tone   
133515             UK news                                                                  Budget 2021: key points at a glance   
5206        Life and style                                  All the gear…: ‘How obsession with kit eclipsed my love of running’   
82510              Society                                             Millions wasted in failed NHS hospital closure programme   
5237                  Food                                                                Maray: restaurant review | Jay Rayner   
77726              Opinion                        How Joe Hildebrand made the murder of a woman about his feelings | Van Badham   
106760            Football                            Football transfer rumours: Barcelona eye Manchester City's Gabriel Jesus?   
47066             Politics                                        EU withdrawal bill needs major rewrites, Lords committee says   
55626              UK news                                         Romanian is second most common non-British nationality in UK   
19766                 News                                                                        Inside the 2 December edition   
144405         Environment                  America’s food deserts start seeing influx of healthy foods thanks to federal funds   
48020              Opinion                                           Don't buy the 'Brexit dividend' myth | David Shariatmadari   
118168          World news                                                          How the right words can end the culture war   
146862          World news                        ‘Astonishing’ errands: Old Enough's Japanese creator on the child star TV hit   
71379             Football                                             Chelsea escape Uefa punishment over ‘antisemitic’ chants   
117475             US news                                   Miami Beach to cut back on famous palm trees over climate concerns   
57619                Books                All Gates Open by Rob Young and Irmin Schmidt review – in praise of the rock band Can   
1449                 Sport                            Have the unstoppable Warriors j